In [29]:
import pandas as pd
import json
import logging
import re
import sys
import pprint
import time
import pkg_resources
import requests
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from grimoirelab_toolkit.datetime import (datetime_to_utc,
                                          str_to_datetime)
from perceval.backends.core.git import GitRepository
from perceval.backends.core.git import Git
from datetime import datetime, timedelta
import pytz
import pandas as pd



In [442]:
#perceval fetching


# url for the git repo to analyze
repo_url = 'https://github.com/elastic/beats.git'
# directory for letting Perceval clone the git repo
repo_dir = '/tmp/beats.git'

# create a Git object, pointing to repo_url, using repo_dir for cloning
repo = Git(uri=repo_url, gitpath=repo_dir)
# fetch all commits as an iteratoir, and iterate it printing each hash
for commit in repo.fetch():
    print(commit)
    break

{'backend_name': 'Git', 'backend_version': '0.10.2', 'perceval_version': '0.12.5', 'timestamp': 1554314148.5678, 'origin': 'https://github.com/elastic/beats.git', 'uuid': 'b4aebd1d4dee681b017c13fdd50675e920890166', 'updated_on': 1391626862.0, 'category': 'commit', 'tag': 'https://github.com/elastic/beats.git', 'data': {'commit': '5b740f90b329476cae70d2103d07239eb0b7f70f', 'parents': [], 'refs': [], 'Author': 'Packetbeat <monica@packetbeat.com>', 'AuthorDate': 'Wed Feb 5 11:01:02 2014 -0800', 'Commit': 'Packetbeat <monica@packetbeat.com>', 'CommitDate': 'Wed Feb 5 11:01:02 2014 -0800', 'message': 'Initial commit', 'files': [{'modes': ['000000', '100644'], 'indexes': ['000000000', '00268614f'], 'action': 'A', 'file': '.gitignore', 'added': '22', 'removed': '0'}, {'modes': ['000000', '100644'], 'indexes': ['000000000', '2e40e099a'], 'action': 'A', 'file': 'LICENSE', 'added': '20', 'removed': '0'}, {'modes': ['000000', '100644'], 'indexes': ['000000000', '1fa0e3c29'], 'action': 'A', 'file'

In [443]:
def author_filter(name):
    pos=name.find('<')
    if pos!=-1:
        a_name=name[:pos]
        
    return a_name
threshold=50
df = []
for commit in repo.fetch():
    try:
        author=author_filter(commit['data']['Author'])
        c_date=datetime_to_utc(datetime.strptime(commit['data']['CommitDate'],'%a %b %d %H:%M:%S %Y %z'))
        df.append({'userid': author,'commitdate':c_date.date()})
    except AttributeError:
        pass
df = pd.DataFrame(df)

In [444]:
df['commitdate'] = df["commitdate"].apply( lambda df : 
datetime(year=df.year, month=df.month, day=df.day))
df['commitdate'] = df['commitdate'].dt.to_period("Q")
df

,commitdate,userid
0,2014Q1,Packetbeat
1,2014Q1,Monica Sarbu
2,2014Q1,Monica Sarbu
3,2014Q1,Monica Sarbu
4,2014Q1,Packetbeat
5,2014Q1,Monica Sarbu
6,2014Q1,Monica Sarbu
7,2014Q1,Monica Sarbu
8,2014Q1,Monica Sarbu
9,2014Q1,Monica Sarbu


In [445]:
dfq=df.groupby(['commitdate','userid']).userid.count().to_frame()
print(dfq)
df_qtr=pd.DataFrame(columns=dfq.index.levels[0],index=dfq.index.levels[1])

                                        userid
commitdate userid                             
2014Q1     Monica Sarbu                     14
           Packetbeat                        2
2014Q2     James Butler                      1
           Josh Boon                         3
           Lewis Marshall                    6
           Marcos Nils                       1
           Marios Nikolaou                   2
           Monica Sarbu                    119
           Packetbeat                       19
           Pradeep Chhetri                   4
           Stéphane Bunel                    3
           午夜                                1
2014Q3     Dan Sowter                        1
           Monica Sarbu                     63
           Packetbeat                        2
           mk2011                            1
2014Q4     Mathieu Lecarme                   1
           Monica Sarbu                      1
           Packetbeat                        1
           Tu

In [446]:
for index, row in dfq.iterrows() :
    df_qtr.loc[index[1]][index[0]]=row[0]
df_qtr=df_qtr.fillna(0).T


In [447]:
df_qtr['total']=df_qtr.sum(axis=1)

In [448]:
df_qtr

userid,2Fast2BCn,40kuai,7lima,AJ,Adi Ben-Dahan,Adrian Serrano,Adrien Grand,Adron Hall,Aidan Rowe,Al Lefebvre,...,whale-shark,wlovins,xy795,yaule,yeer,Álex González,午夜,罗泽轩,韦钰迪,total
commitdate,,,,,,,,,,,,,,,,,,,,,
2014Q1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16
2014Q2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,159
2014Q3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,67
2014Q4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,188
2015Q1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,145
2015Q2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,284
2015Q3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,820
2015Q4,0,0,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,1543
2016Q1,0,0,0,0,0,0,1,6,0,0,...,0,0,0,0,0,0,0,0,0,984


In [449]:
df_qtr=df_qtr.iloc[:,:-1].div(df_qtr["total"], axis=0).mul(100)

In [450]:
df_qtr

userid,2Fast2BCn,40kuai,7lima,AJ,Adi Ben-Dahan,Adrian Serrano,Adrien Grand,Adron Hall,Aidan Rowe,Al Lefebvre,...,wangdisdu,whale-shark,wlovins,xy795,yaule,yeer,Álex González,午夜,罗泽轩,韦钰迪
commitdate,,,,,,,,,,,,,,,,,,,,,
2014Q1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.628931,0.000000,0.000000
2014Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014Q4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015Q1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.121951,0.000000,0.000000,0.000000
2015Q4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.194426,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2016Q1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101626,0.609756,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [451]:
for index,row in df_qtr.iterrows():
    c=0
    for num in df_qtr.loc[index,:].sort_values(ascending=False).cumsum():
        c=c+1
        if num>=threshold:
            df_qtr.loc[index,'wbus_factor']=c
            break
    c=0
    for num in df_qtr.loc[index,:].sort_values(ascending=True).cumsum():
        c=c+1
        if num>=threshold:
            df_qtr.loc[index,'bbus_factor']=c
            break
    

In [452]:
df_qtr

userid,2Fast2BCn,40kuai,7lima,AJ,Adi Ben-Dahan,Adrian Serrano,Adrien Grand,Adron Hall,Aidan Rowe,Al Lefebvre,...,wlovins,xy795,yaule,yeer,Álex González,午夜,罗泽轩,韦钰迪,wbus_factor,bbus_factor
commitdate,,,,,,,,,,,,,,,,,,,,,
2014Q1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,415.0
2014Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.628931,0.000000,0.000000,1.0,415.0
2014Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,415.0
2014Q4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,415.0
2015Q1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,415.0
2015Q2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,415.0
2015Q3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.121951,0.000000,0.000000,0.000000,2.0,413.0
2015Q4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.194426,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,413.0
2016Q1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101626,0.609756,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.0,412.0


es = Elasticsearch(['http://localhost:9200/'])
try:
    es.indices.create('qscore')
except elasticsearch.exceptions.RequestError:
    print('Index already exisits, remove it before running this script again.')
    exit()

In [453]:
for index,rows in df_qtr.iterrows():
    q_score=json.loads(df_qtr.loc[index,:].to_json())
    q_score['quarter']="{}Q{}".format(index.year,index.quarter)
    q_score['project']="elastic beats"
    print(q_score)
    es.index(index='qscore', doc_type='q_score', body=q_score)
    


{'2Fast2BCn ': 0.0, '40kuai ': 0.0, '7lima ': 0.0, 'AJ ': 0.0, 'Adi Ben-Dahan ': 0.0, 'Adrian Serrano ': 0.0, 'Adrien Grand ': 0.0, 'Adron Hall ': 0.0, 'Aidan Rowe ': 0.0, 'Al Lefebvre ': 0.0, 'Alban Mouton ': 0.0, 'Alex K ': 0.0, 'Alex Wormuth ': 0.0, 'Alexander Kazakov ': 0.0, 'Alexander Klizhentas ': 0.0, 'Amanda H. L. de Andrade ': 0.0, 'Amanda H. L. de Andrade Katz ': 0.0, 'Amitosh Swain Mahapatra ': 0.0, 'Andras Kovi ': 0.0, 'Andrew Cholakian ': 0.0, 'Andrew Dunham ': 0.0, 'Andrew Hall ': 0.0, 'Andrew Kroh ': 0.0, 'Andy Georges ': 0.0, 'Antoine Toulme ': 0.0, 'Arkady Emelyanov ': 0.0, 'Armin ': 0.0, 'Armin Braun ': 0.0, 'Arnout Engelen ': 0.0, 'Arsalan Cheema ': 0.0, 'Aryeh Weinreb ': 0.0, 'Aurélien Thieriot ': 0.0, 'Ben Gadbois ': 0.0, 'Benjamin Copeland ': 0.0, 'Benjamin Knofe ': 0.0, 'Berfin Sarı ': 0.0, 'Blerim Sheqa ': 0.0, 'Boaz Shuster ': 0.0, 'Bob Vincent ': 0.0, 'Brandon Morelli ': 0.0, 'Brendan Devenney ': 0.0, 'Brendan LE GLAUNEC ': 0.0, 'Caleb Chaney ': 0.0, 'Calle Pe

{'2Fast2BCn ': 0.0, '40kuai ': 0.0, '7lima ': 0.0, 'AJ ': 0.0, 'Adi Ben-Dahan ': 0.0, 'Adrian Serrano ': 0.0, 'Adrien Grand ': 0.0, 'Adron Hall ': 0.0, 'Aidan Rowe ': 0.0, 'Al Lefebvre ': 0.0, 'Alban Mouton ': 0.0, 'Alex K ': 0.0, 'Alex Wormuth ': 0.0, 'Alexander Kazakov ': 0.0, 'Alexander Klizhentas ': 0.0, 'Amanda H. L. de Andrade ': 0.0, 'Amanda H. L. de Andrade Katz ': 0.0, 'Amitosh Swain Mahapatra ': 0.0, 'Andras Kovi ': 0.0, 'Andrew Cholakian ': 0.0, 'Andrew Dunham ': 0.0, 'Andrew Hall ': 0.0, 'Andrew Kroh ': 13.0265716137, 'Andy Georges ': 0.0, 'Antoine Toulme ': 0.0, 'Arkady Emelyanov ': 0.0, 'Armin ': 0.0, 'Armin Braun ': 0.0, 'Arnout Engelen ': 0.0, 'Arsalan Cheema ': 0.0, 'Aryeh Weinreb ': 0.0, 'Aurélien Thieriot ': 0.064808814, 'Ben Gadbois ': 0.0, 'Benjamin Copeland ': 0.0, 'Benjamin Knofe ': 0.0, 'Berfin Sarı ': 0.0, 'Blerim Sheqa ': 0.0, 'Boaz Shuster ': 0.0, 'Bob Vincent ': 0.0, 'Brandon Morelli ': 0.0, 'Brendan Devenney ': 0.0, 'Brendan LE GLAUNEC ': 0.0, 'Caleb Chaney